# Накопленный охват в динамике по ФО
Пример расчета показателей тематических каналов в разбивке по федеральным округам и месяцам.

## Описание задачи и условий расчета
- Период: 01.01.2022 - 30.09.2022 с разбивкой по месяцам
- Временной интервал: 05:00-29:00
- ЦА: Все 4+ в разбивке по федеральным округам
- Место просмотра: Все места (дом+дача)
- Каналы: .BLACK									
- Статистики: Reach%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
import re
import json
import datetime
import time
import pandas as pd
from IPython.display import JSON

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике телекомпаний найдем нужный нам канал
cats.get_tv_company(name=['.BLACK'])

# id телекомпании .BLACK - 5960.

In [ ]:
# В каталоге демографических переменным найдем переменную Федеральные округа
cats.get_tv_demo_attribute(names=['федерал'])

# Далее в параметры задания зададим ее entityName - federalOkrug

## Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'.
# Минимум 28 дней
date_filter = [('2022-01-01', '2022-09-30')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал - минимум 60 минут
time_filter = None

# Задаем ЦА
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы - телекомпания .BLACK (СЕТЕВОЕ ВЕЩАНИЕ)
company_filter = 'tvCompanyId = 5960'

# Указываем список статистик для расчета
statistics = ['ReachPer']

# Указываем срезы
slices = [
    'federalOkrug', # Федеральный округ 
    'researchMonth' # месяц
]

# Задаем условия сортировки: федеральный округ (от а до я), месяцы (по возр.)
sortings = {'federalOkrug':'ASC', 'researchMonth':'ASC'}

# Задаем опции расчета
options = {
    "kitId": 4, #TV Index Plus All Russia
    "useNbd": False #Расчет накопленного охвата без nbd коррекции
}

## Расчет задания

In [ ]:
# Формируем задание для TV API в формате JSON
task_json = mtask.build_timeband_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                      daytype_filter=daytype_filter, company_filter=company_filter, 
                                      time_filter=time_filter, basedemo_filter=basedemo_filter, 
                                      targetdemo_filter=targetdemo_filter,location_filter=location_filter, 
                                      slices=slices, statistics=statistics, sortings=sortings, options=options)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))

# Приводим порядок столбцов в соответствие с условиями расчета
df = df[slices+statistics]

df

## Настройка внешнего вида таблицы
Пропустите этот шаг, если хотите экспортировать таблицу в ее текущем виде

In [ ]:
# Формируем сводную таблицу: строки - месяц, столбцы - федеральные округа, значения - статистики
df = pd.pivot_table(df, values=statistics,
                        index=['researchMonth'], 
                        columns=['federalOkrug'])
df

In [ ]:
# Поменять местами уровни в столбцах (раскомментируйте при необходимости)
#df = df.swaplevel(0,1, axis=1).sort_index(axis=1)

#df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('thematic_02_reach_by_federal_okrug')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)